In [33]:
class BalanceException(Exception):
    pass

class BankAccount:
    def __init__(self, initial_amount, acct_name, owner_name):
        self.balance = initial_amount
        self.name = acct_name
        self.owner = owner_name
        self.transaction_history = []
        self.record_transaction("Account creation", initial_amount)
        print(f'\nAccount {self.name} created for {self.owner}. Balance = {self.balance:.2f}€')
    
    def get_balance(self):
        print(f'\nAccount {self.name} for {self.owner}. Current balance = {self.balance:.2f}€')
        return self.balance
    
    def record_transaction(self, transaction_type, amount):
        self.transaction_history.append({
            "type": transaction_type,
            "amount": amount,
            "resulting_balance": self.balance
        })
    
    def show_transaction_history(self):
        print(f"\nTransaction history for account {self.name}:")
        for idx, transaction in enumerate(self.transaction_history):
            print(f"{idx+1}. {transaction['type']}: {transaction['amount']:.2f}€ - Balance: {transaction['resulting_balance']:.2f}€")
    
    def deposit(self, amount):
        if amount <= 0:
            print("Deposit amount must be positive")
            return
        
        self.balance += amount
        self.record_transaction("Deposit", amount)
        print(f'Deposit complete. New balance: {self.balance:.2f}€')
        
    def viable_transaction(self, amount):
        if self.balance >= amount:
            return True
        else:
            raise BalanceException(f'Sorry, account {self.name} only has a balance of {self.balance:.2f}€')
    
    def withdraw(self, amount):
        if amount <= 0:
            print("Withdrawal amount must be positive")
            return
            
        try:
            self.viable_transaction(amount)
            self.balance = self.balance - amount
            self.record_transaction("Withdrawal", -amount)
            print('Withdraw complete')
            self.get_balance()
        except BalanceException as error:
            print(f'Withdraw interrupted: {error}')
    
    def transfer(self, amount, account):
        if amount <= 0:
            print("Transfer amount must be positive")
            return
            
        try:
            print('\n*********\n\nBeginning Transfer..🚀')
            self.viable_transaction(amount)
            self.balance -= amount
            self.record_transaction(f"Transfer to {account.name}", -amount)
            account.balance += amount
            account.record_transaction(f"Transfer from {self.name}", amount)
            print(f'\nTransfer of {amount:.2f}€ from {self.name} to {account.name} complete!')
            self.get_balance()
            account.get_balance()
            print('\n********')
        except BalanceException as error:
            print(f'\nTransfer interrupted. {error}')


class InterestRewardsAcct(BankAccount):
    def __init__(self, initial_amount, acct_name, owner_name, interest_rate=0.05):
        super().__init__(initial_amount, acct_name, owner_name)
        self.interest_rate = interest_rate
        
    def deposit(self, amount):
        if amount <= 0:
            print("Deposit amount must be positive")
            return
            
        interest = amount * self.interest_rate
        self.balance = self.balance + amount + interest
        self.record_transaction(f"Deposit with {self.interest_rate*100}% interest", amount + interest)
        print(f'\nDeposit complete with interest bonus of {interest:.2f}€')
        self.get_balance()
    
    def apply_monthly_interest(self):
        interest = self.balance * (self.interest_rate / 12)  # Monthly interest
        self.balance += interest
        self.record_transaction("Monthly interest", interest)
        print(f"Monthly interest of {interest:.2f}€ applied")
        self.get_balance()


class SavingsAcct(InterestRewardsAcct):
    def __init__(self, initial_amount, acct_name, owner_name, interest_rate=0.03, fee=5):
        super().__init__(initial_amount, acct_name, owner_name, interest_rate)
        self.fee = fee
        self.monthly_withdrawals = 0
        self.max_monthly_withdrawals = 3
        
    def withdraw(self, amount):
        if amount <= 0:
            print("Withdrawal amount must be positive")
            return
            
        if self.monthly_withdrawals >= self.max_monthly_withdrawals:
            print(f"\nExceeded maximum monthly withdrawals ({self.max_monthly_withdrawals})")
            return
            
        try:
            total_amount = amount + self.fee
            self.viable_transaction(total_amount)
            self.balance -= total_amount
            self.monthly_withdrawals += 1
            self.record_transaction(f"Withdrawal (with {self.fee}€ fee)", -total_amount)
            print(f'\nWithdraw completed with fee of {self.fee}€.')
            self.get_balance()
            print(f"Remaining monthly withdrawals: {self.max_monthly_withdrawals - self.monthly_withdrawals}")
        except BalanceException as error:
            print(f'\nWithdraw interrupted: {error}')
    
    def reset_monthly_withdrawals(self):
        self.monthly_withdrawals = 0
        print(f"Monthly withdrawal count reset to 0 for account {self.name}")


class CheckingAccount(BankAccount):
    def __init__(self, initial_amount, acct_name, owner_name, overdraft_limit=100):
        super().__init__(initial_amount, acct_name, owner_name)
        self.overdraft_limit = overdraft_limit
        self.overdraft_fee = 20
        
    def viable_transaction(self, amount):
        if self.balance + self.overdraft_limit >= amount:
            return True
        else:
            raise BalanceException(f'Sorry, account {self.name} with overdraft has a limit of {self.balance + self.overdraft_limit:.2f}€')
            
    def withdraw(self, amount):
        if amount <= 0:
            print("Withdrawal amount must be positive")
            return
            
        try:
            self.viable_transaction(amount)
            
            # Apply overdraft fee if necessary
            if amount > self.balance:
                self.balance = self.balance - amount - self.overdraft_fee
                self.record_transaction(f"Withdrawal with overdraft fee", -(amount + self.overdraft_fee))
                print(f'Withdraw complete with overdraft fee of {self.overdraft_fee}€')
            else:
                self.balance = self.balance - amount
                self.record_transaction("Withdrawal", -amount)
                print('Withdraw complete')
                
            self.get_balance()
        except BalanceException as error:
            print(f'Withdraw interrupted: {error}')




In [35]:
# Example usage:
if __name__ == "__main__":
    # Create accounts
    john_checking = CheckingAccount(1000, "Checking", "John Smith")
    john_savings = SavingsAcct(5000, "Savings", "John Smith")
    mary_rewards = InterestRewardsAcct(2000, "Rewards", "Mary Johnson")
    
    # Test operations
    john_checking.deposit(500)
    john_checking.withdraw(1200)  # Should use overdraft
    john_checking.transfer(200, john_savings)
    
    john_savings.deposit(1000)  # Should get interest
    john_savings.withdraw(100)  # Should pay fee
    
    mary_rewards.deposit(500)  # Should get interest
    mary_rewards.apply_monthly_interest()
    
    # Show history
    john_checking.show_transaction_history()


Account Checking created for John Smith. Balance = 1000.00€

Account Savings created for John Smith. Balance = 5000.00€

Account Rewards created for Mary Johnson. Balance = 2000.00€
Deposit complete. New balance: 1500.00€
Withdraw complete

Account Checking for John Smith. Current balance = 300.00€

*********

Beginning Transfer..🚀

Transfer of 200.00€ from Checking to Savings complete!

Account Checking for John Smith. Current balance = 100.00€

Account Savings for John Smith. Current balance = 5200.00€

********

Deposit complete with interest bonus of 30.00€

Account Savings for John Smith. Current balance = 6230.00€

Withdraw completed with fee of 5€.

Account Savings for John Smith. Current balance = 6125.00€
Remaining monthly withdrawals: 2

Deposit complete with interest bonus of 25.00€

Account Rewards for Mary Johnson. Current balance = 2525.00€
Monthly interest of 10.52€ applied

Account Rewards for Mary Johnson. Current balance = 2535.52€

Transaction history for account Ch